In [6]:
from bokeh.io import output_notebook, show, curdoc
from bokeh.layouts import column, row, widgetbox
from bokeh.models import CategoricalColorMapper, Column, ColumnDataSource
from bokeh.models import HoverTool, Select, Slider
from bokeh.palettes import Spectral6
from bokeh.plotting import figure
import numpy as np
import pandas as pd

output_notebook()

Loading BokehJS ...

In [2]:
path = "https://assets.datacamp.com/production/repositories/401/datasets/09378cc53faec573bcb802dce03b01318108a880/gapminder_tidy.csv"
df = pd.read_csv(path)
df.head()

,Country,Year,fertility,life,population,child_mortality,gdp,region
0,Afghanistan,1964,7.671,33.639,10474903.0,339.7,1182.0,South Asia
1,Afghanistan,1965,7.671,34.152,10697983.0,334.1,1182.0,South Asia
2,Afghanistan,1966,7.671,34.662,10927724.0,328.7,1168.0,South Asia
3,Afghanistan,1967,7.671,35.170,11163656.0,323.3,1173.0,South Asia
4,Afghanistan,1968,7.671,35.674,11411022.0,318.1,1187.0,South Asia


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10111 entries, 0 to 10110
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Country          10111 non-null  object 
 1   Year             10111 non-null  int64  
 2   fertility        10100 non-null  float64
 3   life             10111 non-null  float64
 4   population       10108 non-null  float64
 5   child_mortality  9210 non-null   float64
 6   gdp              9000 non-null   float64
 7   region           10111 non-null  object 
dtypes: float64(5), int64(1), object(2)
memory usage: 632.1+ KB


In [9]:
year = 1970
df_ = df[df['Year']==year]
source = ColumnDataSource(data={
    'x': df_['fertility'],
    'y': df_['life'],
    'country': df_['Country'],
    'pop': df_['population'] / 20_000_000 + 2,
    'region': df_['region']
})

color_mapper = CategoricalColorMapper(
    factors=['South Asia', 'Europe & Central Asia',
             'Middle East & North Africa', 'Sub-Saharan Africa', 
             'America', 'East Asia & Pacific'],
    palette=Spectral6
)
hover = HoverTool(tooltips=[('Country', '@country')])
    
def bkapp(doc):
    xmin, xmax = min(source.data['x']), max(source.data['x'])
    ymin, ymax = min(source.data['y']), max(source.data['y'])

    plot = figure(
        title='Gapminder Data for 1970', 
        plot_height=400, 
        plot_width=800,
        x_range=(xmin, xmax),
        y_range=(ymin, ymax)
    )
    plot.circle(x='x', y='y', fill_alpha=0.8, source=source,
                size=8,
                color=dict(field='region', transform=color_mapper),
                legend_field='region')
    plot.xaxis.axis_label = 'Fertility (children per woman)'
    plot.yaxis.axis_label = 'Life Expectancy (years)'
    plot.legend.location = 'top_right'
    plot.add_tools(hover)


    def update_plot(attr, old, new):
        yr = slider.value
        x = x_select.value
        y = y_select.value
        data_new = {
            'x': df[df['Year']==yr][x],
            'y': df[df['Year']==yr][y],
            'country': df[df['Year']==yr]['Country'],
            'pop': df[df['Year']==yr]['population'] / 20_000_000 + 2,
            'region': df[df['Year']==yr]['region']
        }
        source.data = data_new
        plot.x_range.start = min(df[x])
        plot.x_range.end = max(df[x])
        plot.y_range.start = min(df[y])
        plot.y_range.end = max(df[y])
        plot.title.text = f"Gapminder data for {yr}"


    slider = Slider(start=1970, end=2010, step=1, value=1970, title='Year')
    slider.on_change('value', update_plot)
    
    x_select = Select(
        options=['fertility', 'life', 'child_mortality', 'gdp'],
        value='fertility',
        title='x-axis data'
    )
    x_select.on_change('value', update_plot)
    
    y_select = Select(
        options=['fertility', 'life', 'child_mortality', 'gdp'],
        value='life',
        title='y-axis data'
    )
    y_select.on_change('value', update_plot)
    
    doc.add_root(row(Column(slider, x_select, y_select), plot))

show(bkapp)